In [1]:
import os
import pandas as pd
import numpy as np
import torch
import matplotlib as plt
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from torch.utils.data import TensorDataset, random_split

from torch.utils.data import Dataset, DataLoader

import psutil

In [2]:
# IF THE TNESOR IS ALREADY DOWNLOADED, SKIP OVER ALL OF THIS UNTIL YOU FIND THE CELL USING THE LOAD FUNCTION

# Dictionary of file names cateogrized by wich round they are in
files_by_round = {'r0': ['2024-06-05_r0_np-cc_repeat_aa-seq-counts.tsv.gz', '2024-06-05_r0_np-cc_aa-seq-counts.tsv.gz', '2024-06-05_r0_np-cc_gel-extract_aa-seq-counts.tsv.gz'], 'r1': ['2024-06-05_r1_np-cc_gel-extract_aa-seq-counts.tsv.gz', '2024-06-05_r1_np-cc_aa-seq-counts.tsv.gz', ], 'r5': ['2024-06-05_r5_np-cc_aa-seq-counts.tsv.gz']}

# files_by_round = {'r0': ['2024-06-05_r0_np-cc_repeat_aa-seq-counts.tsv.gz', '2024-06-05_r0_np-cc_aa-seq-counts.tsv.gz']}
# /Users/tlorant/Documents/MBE/environment.yml
r0 = pd.DataFrame()
r1 = pd.DataFrame()
r5 = pd.DataFrame()

for round in files_by_round:
    for file in files_by_round[round]:
        print(file)
        if round == 'r0':
            if r0.size == 0:
                r0 = pd.read_csv("./../out/corrected/counts/" + file, sep='\t')
            else:
                new_read = pd.read_csv("./../out/corrected/counts/" + file, sep='\t')
                r0 = pd.merge(r0, new_read, how='outer', on='sequence', suffixes=('_r0', '_new_read'))
                r0.fillna(0, inplace=True)
                r0['count'] = r0['count_r0'] + r0['count_new_read']
                r0.drop(columns=['count_r0', 'count_new_read'], axis=1, inplace=True)

        if round == 'r1':
            if r1.size == 0:
                r1 = pd.read_csv("./../out/corrected/counts/" + file, sep='\t')
            else:
                new_read = pd.read_csv("./../out/corrected/counts/" + file, sep='\t')
                r1 = pd.merge(r1, new_read, how='outer', on='sequence', suffixes=('_r1', '_new_read'))
                r1.fillna(0, inplace=True)
                r1['count'] = r1['count_r1'] + r1['count_new_read']
                r1.drop(columns=['count_r1', 'count_new_read'], axis=1, inplace=True)

        if round == 'r5':
            if r5.size == 0:
                r5 = pd.read_csv("./../out/corrected/counts/" + file, sep='\t')
            else:
                new_read = pd.read_csv("./../out/corrected/counts/" + file, sep='\t')
                r5 = pd.merge(r5, new_read, how='outer', on='sequence', suffixes=('_r5', '_new_read'))
                r5.fillna(0, inplace=True)
                r5['count'] = r5['count_r0'] + r5['count_new_read']
                r5.drop(columns=['count_r0', 'count_new_read'], axis=1, inplace=True)

2024-06-05_r0_np-cc_repeat_aa-seq-counts.tsv.gz
2024-06-05_r0_np-cc_aa-seq-counts.tsv.gz
2024-06-05_r0_np-cc_gel-extract_aa-seq-counts.tsv.gz
2024-06-05_r1_np-cc_gel-extract_aa-seq-counts.tsv.gz
2024-06-05_r1_np-cc_aa-seq-counts.tsv.gz
2024-06-05_r5_np-cc_aa-seq-counts.tsv.gz


In [3]:
# merge r0 and r1
r01 = pd.merge(r0, r1, on='sequence', how='outer', suffixes=('_r0', '_r1'))
r01.fillna(0, inplace=True)
r01 = r01[:10000]

In [4]:
del files_by_round
del file
del round
del new_read

In [5]:
# Get the amion acids
# aminon_acids = pd.Series(list(lil['sequence'][0])).unique()
aminon_acids = pd.Series(list(r01['sequence'][0])).unique()
print(aminon_acids)
print(len(aminon_acids))

['M' 'A' 'D' 'G' 'Y' 'L' 'P' 'W' 'E' 'N' 'C' 'I' 'R' 'K' 'Q' 'H' 'V' 'F'
 'S' 'T' '*']
21


In [6]:
aa_mapping = {aminon_acids[i]: i for i in range(len(aminon_acids))}
aa_mapping

{'M': 0,
 'A': 1,
 'D': 2,
 'G': 3,
 'Y': 4,
 'L': 5,
 'P': 6,
 'W': 7,
 'E': 8,
 'N': 9,
 'C': 10,
 'I': 11,
 'R': 12,
 'K': 13,
 'Q': 14,
 'H': 15,
 'V': 16,
 'F': 17,
 'S': 18,
 'T': 19,
 '*': 20}

In [7]:
aa_mapping['Nan'] = 21
aa_mapping

{'M': 0,
 'A': 1,
 'D': 2,
 'G': 3,
 'Y': 4,
 'L': 5,
 'P': 6,
 'W': 7,
 'E': 8,
 'N': 9,
 'C': 10,
 'I': 11,
 'R': 12,
 'K': 13,
 'Q': 14,
 'H': 15,
 'V': 16,
 'F': 17,
 'S': 18,
 'T': 19,
 '*': 20,
 'Nan': 21}

In [8]:
token_seq = r01['sequence'].apply(lambda x: pd.Series(list(x)))
token_seq

,0,1,2,3,4,5,6,7,8,9,...,730,731,732,733,734,735,736,737,738,739
0,M,A,A,D,G,Y,L,P,D,W,...,R,Y,L,T,R,N,L,*,NaN,NaN
1,M,A,A,D,G,Y,L,P,D,W,...,T,R,Y,L,T,R,N,L,*,NaN
2,M,A,A,D,G,Y,L,P,D,W,...,R,Y,L,T,R,N,L,*,NaN,NaN
3,M,A,A,D,G,Y,L,P,D,W,...,L,T,R,N,L,*,NaN,NaN,NaN,NaN
4,M,A,A,D,G,Y,L,P,D,W,...,R,Y,L,T,R,N,L,*,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,M,A,A,D,G,Y,L,P,D,W,...,L,T,R,N,L,*,NaN,NaN,NaN,NaN
9996,M,A,A,D,G,Y,L,P,D,W,...,R,Y,L,T,R,N,L,*,NaN,NaN
9997,M,A,A,D,G,Y,L,P,D,W,...,L,T,R,N,L,*,NaN,NaN,NaN,NaN
9998,M,A,A,D,G,Y,L,P,D,W,...,T,R,N,L,*,NaN,NaN,NaN,NaN,NaN


In [9]:
token_seq.columns = [f'aa_{i}' for i in range(token_seq.shape[1])]
token_seq

,aa_0,aa_1,aa_2,aa_3,aa_4,aa_5,aa_6,aa_7,aa_8,aa_9,...,aa_730,aa_731,aa_732,aa_733,aa_734,aa_735,aa_736,aa_737,aa_738,aa_739
0,M,A,A,D,G,Y,L,P,D,W,...,R,Y,L,T,R,N,L,*,NaN,NaN
1,M,A,A,D,G,Y,L,P,D,W,...,T,R,Y,L,T,R,N,L,*,NaN
2,M,A,A,D,G,Y,L,P,D,W,...,R,Y,L,T,R,N,L,*,NaN,NaN
3,M,A,A,D,G,Y,L,P,D,W,...,L,T,R,N,L,*,NaN,NaN,NaN,NaN
4,M,A,A,D,G,Y,L,P,D,W,...,R,Y,L,T,R,N,L,*,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,M,A,A,D,G,Y,L,P,D,W,...,L,T,R,N,L,*,NaN,NaN,NaN,NaN
9996,M,A,A,D,G,Y,L,P,D,W,...,R,Y,L,T,R,N,L,*,NaN,NaN
9997,M,A,A,D,G,Y,L,P,D,W,...,L,T,R,N,L,*,NaN,NaN,NaN,NaN
9998,M,A,A,D,G,Y,L,P,D,W,...,T,R,N,L,*,NaN,NaN,NaN,NaN,NaN


In [10]:
seq_mapped = token_seq.map(lambda x: aa_mapping.get(x))
seq_mapped

,aa_0,aa_1,aa_2,aa_3,aa_4,aa_5,aa_6,aa_7,aa_8,aa_9,...,aa_730,aa_731,aa_732,aa_733,aa_734,aa_735,aa_736,aa_737,aa_738,aa_739
0,0,1,1,2,3,4,5,6,2,7,...,12,4,5,19,12,9.0,5.0,20.0,NaN,NaN
1,0,1,1,2,3,4,5,6,2,7,...,19,12,4,5,19,12.0,9.0,5.0,20.0,NaN
2,0,1,1,2,3,4,5,6,2,7,...,12,4,5,19,12,9.0,5.0,20.0,NaN,NaN
3,0,1,1,2,3,4,5,6,2,7,...,5,19,12,9,5,20.0,NaN,NaN,NaN,NaN
4,0,1,1,2,3,4,5,6,2,7,...,12,4,5,19,12,9.0,5.0,20.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,1,1,2,3,4,5,6,2,7,...,5,19,12,9,5,20.0,NaN,NaN,NaN,NaN
9996,0,1,1,2,3,4,5,6,2,7,...,12,4,5,19,12,9.0,5.0,20.0,NaN,NaN
9997,0,1,1,2,3,4,5,6,2,7,...,5,19,12,9,5,20.0,NaN,NaN,NaN,NaN
9998,0,1,1,2,3,4,5,6,2,7,...,19,12,9,5,20,NaN,NaN,NaN,NaN,NaN


In [11]:
seq_mapped.fillna(21, inplace=True)
seq_mapped

,aa_0,aa_1,aa_2,aa_3,aa_4,aa_5,aa_6,aa_7,aa_8,aa_9,...,aa_730,aa_731,aa_732,aa_733,aa_734,aa_735,aa_736,aa_737,aa_738,aa_739
0,0,1,1,2,3,4,5,6,2,7,...,12,4,5,19,12,9.0,5.0,20.0,21.0,21.0
1,0,1,1,2,3,4,5,6,2,7,...,19,12,4,5,19,12.0,9.0,5.0,20.0,21.0
2,0,1,1,2,3,4,5,6,2,7,...,12,4,5,19,12,9.0,5.0,20.0,21.0,21.0
3,0,1,1,2,3,4,5,6,2,7,...,5,19,12,9,5,20.0,21.0,21.0,21.0,21.0
4,0,1,1,2,3,4,5,6,2,7,...,12,4,5,19,12,9.0,5.0,20.0,21.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,1,1,2,3,4,5,6,2,7,...,5,19,12,9,5,20.0,21.0,21.0,21.0,21.0
9996,0,1,1,2,3,4,5,6,2,7,...,12,4,5,19,12,9.0,5.0,20.0,21.0,21.0
9997,0,1,1,2,3,4,5,6,2,7,...,5,19,12,9,5,20.0,21.0,21.0,21.0,21.0
9998,0,1,1,2,3,4,5,6,2,7,...,19,12,9,5,20,21.0,21.0,21.0,21.0,21.0


In [12]:
print(f"Available memory: { psutil.virtual_memory().available / (1024 ** 3):.2f} GB")

Available memory: 351.25 GB


In [13]:
del token_seq
del aminon_acids
del r0
del r1
del r5

In [14]:
print(f"Available memory: { psutil.virtual_memory().available / (1024 ** 3):.2f} GB")

Available memory: 351.25 GB


In [15]:
import torch.nn.functional as F
seq_tensor = torch.tensor(seq_mapped.values)
print(type(seq_tensor))
long_seq_tensor = seq_tensor.long()
one_hot1 = F.one_hot(long_seq_tensor, num_classes=len(aa_mapping))
one_hot1

<class 'torch.Tensor'>


tensor([[[1, 0, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 1, 0],
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1]],

        [[1, 0, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 1, 0],
         [0, 0, 0,  ..., 0, 0, 1]],

        [[1, 0, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 1, 0],
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1]],

        ...,

        [[1, 0, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1]],

        [[1, 0, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0],
         [0,

In [16]:
one_hot1.shape

torch.Size([10000, 740, 22])

In [17]:
del long_seq_tensor
del seq_mapped
del seq_tensor

In [18]:
one_hot1.shape[1]

740

In [19]:
one_hot1_2d = one_hot1.view(one_hot1.shape[0], one_hot1.shape[1] * one_hot1.shape[2])
one_hot1_2d

tensor([[1, 0, 0,  ..., 0, 0, 1],
        [1, 0, 0,  ..., 0, 0, 1],
        [1, 0, 0,  ..., 0, 0, 1],
        ...,
        [1, 0, 0,  ..., 0, 0, 1],
        [1, 0, 0,  ..., 0, 0, 1],
        [1, 0, 0,  ..., 0, 0, 1]])

In [20]:
one_hot1_2d.shape

torch.Size([10000, 16280])

In [21]:
r0_counts = (torch.tensor(r01['count_r0'].values)).view(-1,1)
r0_counts

tensor([[1.],
        [1.],
        [0.],
        ...,
        [0.],
        [0.],
        [1.]], dtype=torch.float64)

In [22]:
r1_counts = (torch.tensor(r01['count_r1'].values)).view(-1,1)
r1_counts

tensor([[0.],
        [0.],
        [1.],
        ...,
        [1.],
        [1.],
        [0.]], dtype=torch.float64)

In [23]:
print(r0_counts.shape)
print(r1_counts.shape)

torch.Size([10000, 1])
torch.Size([10000, 1])


In [24]:
type(r0_counts)

torch.Tensor

In [81]:
r0_sum = torch.sum(r0_counts)
r1_sum = torch.sum(r1_counts)

In [26]:
r0_counts = r0_counts/r0_sum
r1_counts = r1_counts/r1_sum

In [27]:
import gc

del one_hot1
del r01
del r0_sum
# del r1_sum

gc.collect()

0

In [28]:
# import psutil

print(f"Available memory: { psutil.virtual_memory().available / (1024 ** 3):.2f} GB")
# last memory 48.26 GB
# last 186.45

Available memory: 350.08 GB


In [29]:
print(one_hot1_2d.shape[0])
r1_counts.shape[0]

10000


10000

In [30]:
with torch.no_grad():
    data_tensor = torch.cat((r0_counts, r1_counts, one_hot1_2d), dim=1)

In [31]:
data_tensor.shape

torch.Size([10000, 16282])

In [32]:
torch.save(data_tensor, 'stored_data_tensor.pt')

In [33]:
# data_tensor = torch.tensor((torch.load('stored_data_tensor.pt')))

In [34]:
data_tensor

tensor([[1.6250e-04, 0.0000e+00, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         1.0000e+00],
        [1.6250e-04, 0.0000e+00, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         1.0000e+00],
        [0.0000e+00, 1.4495e-04, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         1.0000e+00],
        ...,
        [0.0000e+00, 1.4495e-04, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         1.0000e+00],
        [0.0000e+00, 1.4495e-04, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         1.0000e+00],
        [1.6250e-04, 0.0000e+00, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         1.0000e+00]], dtype=torch.float64)

In [35]:
# Start desigining the model

# Split the data:
# from torch.utils.data import TensorDataset, random_split
######################################
# if type(data_tensor) == pd.DataFrame:
#     data_tensor = torch.tensor(data_tensor.values)

# dataset = TensorDataset(data_tensor)

In [36]:
training_data, validation_data = train_test_split(data_tensor, train_size=0.8, test_size=0.2, shuffle=True, random_state=42)

In [37]:
class AADataset(Dataset):
    
    def __init__(self, data):
        super().__init__()
        
        self.sample_size = (data.size())[1]

        self.indep = torch.cat((data[:,:1], data[:,2:]), dim = 1)
        self.dep = data[:,1]
        

    def __getitem__(self, index):
        return self.indep[index], self.dep[index]

    def __len__(self):
        return self.sample_size
    
    def get_cols(self):
        return self.indep.shape[1]

In [38]:
t_ds = AADataset(training_data)
v_ds = AADataset(validation_data)

In [39]:
t_dl = DataLoader(t_ds, batch_size=32)
v_dl = DataLoader(v_ds, batch_size=32)

In [40]:
# I don't think that you need iterators, I think you can do it without 
t_iter = iter(t_dl)
v_iter = iter(v_dl)


In [41]:
# # Determine the sizes for the splits
# total_size = len(dataset)
# split_80 = int(total_size * 0.8)
# split_20 = total_size - split_80

# # Split the dataset
# train_dataset, validation_dataset = random_split(dataset, [split_80, split_20])

# # Convert the subsets back to tensors
# train_data = torch.stack([x[0] for x in train_dataset])
# validation_data = torch.stack([x[0] for x in validation_dataset])

In [42]:
# In theory you shouldn't need this...

# del dataset
# del split_20
# del split_80
# del data_tensor
# del train_dataset
# del validation_dataset


In [43]:
# t_indep = torch.cat((train_data[:,:1], train_data[:,2:]), dim = 1)
# v_indep = torch.cat((validation_data[:,:1], validation_data[:,2:]), dim = 1)

# t_dep = train_data[:,1]
# v_dep = validation_data[:,1]

In [44]:
n_coeffs =t_ds.get_cols()
n_coeffs

16281

In [45]:
# coeffs = torch.rand(n_coeffs)-0.5
# coeffs

In [46]:
# t_iter = iter(t_dl)
# while True:
#     try: 
#         b = next(t_iter)
#         print(b)
#     except IndexError:
#         break

In [47]:
# Function to calcualte predictions
def calc_preds(coeffs, indeps): 
    return (indeps*coeffs).sum(axis=1) # axis = 1 because we want to sum the rows 

# Function to calcualte loss:
def calc_loss(coeffs, indeps, deps): 
    return torch.abs(calc_preds(coeffs, indeps)-deps).mean()

In [59]:
def one_batch(batch, coeffs, lr):
    loss = calc_loss(coeffs, batch[0], batch[1])
    loss.backward() # Calculates the graident
    with torch.no_grad(): update_coeffs(coeffs, lr)
    return loss

def update_coeffs(coeffs, lr):
    coeffs.sub_(coeffs.grad * lr)
    coeffs.grad.zero_()

def one_epoch(coeffs, lr):
    loss = 0
    t_iter = iter(t_dl)
    while True:
        try:
            b = next(t_iter)
            loss = one_batch(b, coeffs, lr)
        except IndexError:
            break
    # for b in enumerate(t_dl): 
    #     print("batch: ", b)
    #     loss = one_batch(b, coeffs, lr)
        
    # loss = calc_loss(coeffs, dataset.t_indep, dataset.t_dep)
    # loss.backward() # Calculates the graident
    # with torch.no_grad(): update_coeffs(coeffs, lr)
    print(f"{loss:.3f}", end="; ")

def init_coeffs(): return (torch.rand(n_coeffs)-0.5).requires_grad_()

In [60]:
def train_model(epochs=30, lr=0.01):
    torch.manual_seed(442)
    coeffs = init_coeffs()

    for i in range(epochs):
        one_epoch(coeffs, lr=lr)
    return coeffs

In [70]:
# t_iter = iter(t_dl)
# v_iter = iter(v_dl)
coeffs = train_model(30, 0.001)

1.561; 1.061; 0.800; 0.683; 0.625; 0.582; 0.556; 0.524; 0.502; 0.485; 0.465; 0.440; 0.421; 0.402; 0.384; 0.379; 0.364; 0.349; 0.343; 0.327; 0.316; 0.305; 0.301; 0.289; 0.278; 0.263; 0.251; 0.243; 0.241; 0.235; 

In [71]:
coeffs

tensor([-0.4629,  0.1523,  0.2409,  ...,  0.3359,  0.3033, -0.2650],
       requires_grad=True)

In [72]:
# To test the accuracy using the validation dataset
val_preds = calc_preds(coeffs, v_ds.indep)
val_preds

tensor([0.1473, 0.2787, 0.4305,  ..., 0.0882, 0.1241, 0.1938],
       dtype=torch.float64, grad_fn=<SumBackward1>)

In [73]:
diff = torch.abs(val_preds - v_ds.dep)
diff

tensor([0.1470, 0.2786, 0.4304,  ..., 0.0882, 0.1241, 0.1938],
       dtype=torch.float64, grad_fn=<AbsBackward0>)

In [82]:
sum_avg = (torch.sum(diff))/r1_sum

In [83]:
sum_avg

tensor(712.6857, dtype=torch.float64, grad_fn=<DivBackward0>)